In [25]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolio
import dataAck
import warnings
warnings.filterwarnings("ignore")
import portfolioGeneration
from IPython.display import display
import datetime 
startTime = datetime.datetime.now()

In [26]:
modelHashes = portfolioGeneration.getAllPortfolioModels()

In [27]:
allModels = portfolio.getModelsByKey(modelHashes)
modelsSeen = len(allModels)

In [28]:
joinedData = portfolioGeneration.getPertinentDataForModels(allModels)

((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150)
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50)
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25)
((10, ('IWB', 16, None, None, 1), 15, 'VNQ'), 10, 25)
((10, ('FXE', 27, 8, None, 3), 7, 'RSP'), 10, 25)
((5, ('SLV', 26, None, 10, 2), 10, 'EWW'), 5, 50)
((22, ('EFA', 9, 6, None, 1), 7, 'VNQ'), 22, 50)
((44, ('LQD', 23, None, None, 3), 15, 'VGT'), 44, 100)
((5, ('XBI', 12, None, 9, 2), 15, 'IVE'), 5, 25)
((22, ('FXE', 27, 8, None, 3), 7, 'RSP'), 22, 100)
((5, ('XBI', 27, 1, None, 3), 10, 'VTV'), 5, 25)
((44, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 44, 25)
((5, ('SDY', 19, 18, None, 1), 7, 'IYR'), 5, 100)
((22, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 22, 25)
((10, ('IWB', 42, None, 13, 1), 10, 'LQD'), 10, 150)
((10, ('IWR', 4, None, None, 1), 10, 'SLV'), 10, 25)
((22, ('IWB', 42, None, 13, 1), 10, 'LQD'), 22, 25)
((44, ('EFA', 9, 6, None, 1), 7, 'VNQ'), 44, 100)
((5, ('GLD', 36, None, 11, 1), 10, 'XLV'), 5, 300)
((5, ('XBI', 9, None, None, 1), 1

In [29]:
##SHOULD BE NAN IN MANY SERIES...DON'T WANT TO UNNECESSARILY TRUNCATE
display(joinedData)

,Adj_Close_XLV,Adj_Close_GLD,Adj_Close_LQD,Adj_Close_VBR,Adj_Close_EEM,Adj_Close_KBE,Adj_Close_VNQ,Adj_Close_IWB,Adj_Close_RSP,Adj_Close_FXE,Adj_Close_EWW,Adj_Close_SLV,Adj_Close_EFA,Adj_Close_VGT,Adj_Close_IVE,Adj_Close_XBI,Adj_Close_VTV,Adj_Close_IYR,Adj_Close_SDY,Adj_Close_IWR
Date,,,,,,,,,,,,,,,,,,,,
1996-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.004667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.921935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.958705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.958705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.808838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.808838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.885165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.765663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996-04-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.160939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
portfolioGeneration.generateRawPredictionsMP(allModels, joinedData, threadsToUse=3)

((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50) 1.0 2017-08-17 00:00:00
((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150) 1.0 2017-08-17 00:00:00
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25) 1.0 2017-08-17 00:00:00
((10, ('IWB', 16, None, None, 1), 15, 'VNQ'), 10, 25) 1.0 2017-08-17 00:00:00
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50) 1.0 2017-08-09 00:00:00
((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150) -1.0 2017-08-09 00:00:00
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25) 1.0 2017-07-28 00:00:00
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50) 1.0 2017-08-10 00:00:00
((10, ('IWB', 16, None, None, 1), 15, 'VNQ'), 10, 25) 1.0 2017-07-28 00:00:00
((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150) -1.0 2017-08-10 00:00:00
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25) -1.0 2017-07-31 00:00:00
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50) 1.0 2017-08-11 00:00:00
((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150) 1.0 2017-08-11 00:00:00
((10, ('KBE', 29

True

In [31]:
##UPLOAD MODEL PREDICTIONS

In [32]:
##STORE TODAY AGGREGATE FOR ALL MODELS
def produceModelPredictions(allModels, joinedData):
    for model in allModels:
        todayAggregatePrediction = portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        print(model.describe(), todayAggregatePrediction)
        dayToUpload = portfolio.getToday()
        if dayToUpload == joinedData.index[-1]:
            print("SKIPPING UPLOAD, MUST WAIT UNTIL TOMORROW")
            continue
        portfolio.storeAggregateModelPrediction(model,\
                            todayAggregatePrediction, portfolio.getToday(), shouldReturn = False)

In [33]:
produceModelPredictions(allModels, joinedData)

((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150) 0.42857142857142855
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50) 0.7142857142857143
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25) 0.4666666666666667
((10, ('IWB', 16, None, None, 1), 15, 'VNQ'), 10, 25) 0.7333333333333333
((10, ('FXE', 27, 8, None, 3), 7, 'RSP'), 10, 25) 1.0
((5, ('SLV', 26, None, 10, 2), 10, 'EWW'), 5, 50) 0.6
((22, ('EFA', 9, 6, None, 1), 7, 'VNQ'), 22, 50) 1.0
((44, ('LQD', 23, None, None, 3), 15, 'VGT'), 44, 100) -0.2
((5, ('XBI', 12, None, 9, 2), 15, 'IVE'), 5, 25) 0.7333333333333333
((22, ('FXE', 27, 8, None, 3), 7, 'RSP'), 22, 100) 1.0
((5, ('XBI', 27, 1, None, 3), 10, 'VTV'), 5, 25) 0.6
((44, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 44, 25) 1.0
((5, ('SDY', 19, 18, None, 1), 7, 'IYR'), 5, 100) -0.14285714285714285
((22, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 22, 25) 1.0
((10, ('IWB', 42, None, 13, 1), 10, 'LQD'), 10, 150) 1.0
((10, ('IWR', 4, None, None, 1), 10, 'SLV'), 10, 25) -0.4
((22, ('IWB', 42, None, 13,

In [34]:
##UPLOAD PORTFOLIO PREDICTIONS

In [35]:
allPortfolios = portfolio.getPortfolios()
portfoliosSeen = len(allPortfolios)

In [36]:
modelsInPortfolio = {}
portfolioTypes = {}
for portfolioInfo in allPortfolios:
    print(portfolioInfo)
    models = portfolio.getModelsByKey(portfolio.getPortfolioModels(portfolioInfo["key"]))
    modelsInPortfolio[portfolioInfo["key"]] = models
    portfolioTypes[portfolioInfo["key"]] = portfolioInfo["portfolioType"]
    
    for model in models:
        print(model.describe())

{'key': '443958fe526d5b10a120afb1acf292b2e21a9c112da6aa22bfd89d09', 'description': 'Small Portfolio with Unique Models', 'benchmark': 'SPY', 'portfolioType': 'EW By Ticker'}
((22, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 22, 25)
((10, ('FXE', 27, 8, None, 3), 7, 'RSP'), 10, 25)
((5, ('XBI', 12, None, 9, 2), 15, 'IVE'), 5, 25)
((5, ('SDY', 19, 18, None, 1), 7, 'IYR'), 5, 100)
((10, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 10, 25)
((44, ('LQD', 23, None, None, 3), 15, 'VGT'), 44, 100)
((10, ('IWB', 42, None, 13, 1), 10, 'LQD'), 10, 150)
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50)
((10, ('IWB', 16, None, None, 1), 15, 'VNQ'), 10, 25)
((10, ('IWR', 4, None, None, 1), 10, 'SLV'), 10, 25)
((44, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 44, 25)
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25)
((5, ('XBI', 9, None, None, 1), 10, 'VGT'), 5, 25)
((44, ('EFA', 9, 6, None, 1), 7, 'VNQ'), 44, 100)
((10, ('GLD', 36, None, 11, 1), 7, 'XLV'), 10, 150)
((22, ('EFA', 9, 6, None, 1), 7, 'VNQ'), 22, 50)
((5, ('GLD',

In [37]:
print(allModels)

[<dataAck.algoBlob object at 0x7fb32d548b70>, <dataAck.algoBlob object at 0x7fb32d548940>, <dataAck.algoBlob object at 0x7fb32d548978>, <dataAck.algoBlob object at 0x7fb32d54e438>, <dataAck.algoBlob object at 0x7fb32d54e5c0>, <dataAck.algoBlob object at 0x7fb32d54e470>, <dataAck.algoBlob object at 0x7fb32d54e2b0>, <dataAck.algoBlob object at 0x7fb32d5737b8>, <dataAck.algoBlob object at 0x7fb32d573908>, <dataAck.algoBlob object at 0x7fb32d573a58>, <dataAck.algoBlob object at 0x7fb32d573ba8>, <dataAck.algoBlob object at 0x7fb32d573cf8>, <dataAck.algoBlob object at 0x7fb32d573e48>, <dataAck.algoBlob object at 0x7fb32d573f60>, <dataAck.algoBlob object at 0x7fb32d573ef0>, <dataAck.algoBlob object at 0x7fb32d57b278>, <dataAck.algoBlob object at 0x7fb32d57b3c8>, <dataAck.algoBlob object at 0x7fb32d57b518>, <dataAck.algoBlob object at 0x7fb32d57b668>, <dataAck.algoBlob object at 0x7fb32d57b7b8>, <dataAck.algoBlob object at 0x7fb32d57b908>]


In [38]:
aggregateReturns, aggregatePredictions = portfolioGeneration.generateAggregateReturnsPredictions(allModels, joinedData)

In [39]:
display(aggregatePredictions)

,2faf49abf1fbcae37bc81114b876dc2cffa988e43c9810c0401c755a,fce63ca45cf23a15a747a745bb5abc7e4dd97ec9a9c7b2e203977006,98f0c247c49553f0637560916983adb0645ada412ca8548f7ab70f89,8f9d61400bc832f5b1af1cbac268de277f955a2aad6ae71c3e7fc4da,759581b497bcd8f07c66fede156ce5c6d1fe826b0192a98f030742a7,851fd8e7b78397eaa2cabf4c630f1c9241a65638c1c8670d34dabcfd,6b7dc2dcdc33c1821c5c7fefcc9aa5857a8af3da2e9a65781f5645ad,3df5c4064d67b85ba4424dcc5a0a1f0e260b36cc5096855479beed22,7d0babac85bd5240804bca4fd99b925e30f55a7d299076ee21ce25ee,6aa37ddc5e5213a1d76ec495a4fc9207c5fac29cdf5db2f5547ca08f,...,bc43e4c9a00c0e3749c0ba7906dc31a0bda35e01f302abf87aeb7d7a,b6bfc86cdf3ba24ff32db7f039190bace80978bb9c1b8fddae4175ad,e5b77ac840e0694f124e4ceb80219238646ec98c98391057e8b5095e,c155c65771efa17daaafbd3ee1c327d5b1843eade842874a01566d43,43f88763edefd7af535eb11152055d8f83a70858286b6daa4319b127,4d69d02e283395358524757cf4ad40ffb4d5a1629a0591aa402fb5d1,bfb69268004322605e67a1d50f2565c8475076e986b2f0a53c76ad45,525cf44e8172173b3f57b5f8b993b990f74dbe4f6ebf25cee659edc8,2083db1aa7ff59b8962603994801edc673c219bc1d1dfbd623522aaa,de51d3387dba4ff1c7191d771fff8acb8c714379633023cd249c2d0e
2009-01-16,-0.453125,0.500000,0.606689,-0.442749,0.37500,0.001953,-0.093750,0.191895,-0.374512,0.015625,...,0.269836,0.140625,0.326050,0.388672,0.195312,0.060547,0.156250,-0.095703,0.828125,0.143250
2009-01-20,-0.359375,0.015625,0.599854,-0.448853,0.31250,0.000000,-0.156250,0.148865,-0.382141,0.015625,...,0.258118,0.171875,0.310425,0.429688,0.201172,0.074219,0.062500,-0.134766,0.839844,0.121704
2009-01-21,-0.281250,0.000000,0.593262,-0.452515,0.21875,-0.001953,-0.218750,0.105774,-0.388428,0.031250,...,0.245117,0.125000,0.294922,0.470703,0.205078,0.093750,0.031250,-0.167969,0.863281,0.100159
2009-01-22,-0.218750,0.015625,0.586914,-0.454346,0.15625,-0.001953,-0.187500,0.062744,-0.393677,0.062500,...,0.230591,0.093750,0.278992,0.505859,0.199219,0.113281,-0.031250,-0.197266,0.867188,0.078857
2009-01-23,-0.140625,0.046875,0.579956,-0.454468,0.15625,-0.005859,-0.250000,0.019836,-0.397583,0.093750,...,0.216064,0.093750,0.263367,0.548828,0.205078,0.134766,-0.031250,-0.218750,0.871094,0.058472
2009-01-26,-0.093750,0.015625,0.573242,-0.452393,0.12500,-0.003906,-0.281250,-0.022522,-0.400269,0.140625,...,0.200378,0.046875,0.247986,0.585938,0.195312,0.160156,-0.062500,-0.238281,0.875000,0.038208
2009-01-27,-0.093750,0.062500,0.566772,-0.448608,0.09375,-0.003906,-0.234375,-0.064209,-0.401489,0.203125,...,0.183472,0.000000,0.232117,0.621094,0.179688,0.189453,-0.125000,-0.255859,0.875000,0.018066
2009-01-28,-0.093750,0.093750,0.559814,-0.443359,0.12500,-0.005859,-0.296875,-0.105042,-0.401489,0.265625,...,0.167114,0.031250,0.216858,0.658203,0.167969,0.216797,-0.109375,-0.261719,0.867188,-0.000854
2009-01-29,-0.078125,0.093750,0.552979,-0.436035,0.15625,-0.005859,-0.359375,-0.144470,-0.400085,0.328125,...,0.149658,-0.031250,0.201782,0.691406,0.154297,0.250000,-0.093750,-0.261719,0.855469,-0.019653
2009-01-30,-0.109375,0.156250,0.546631,-0.427246,0.15625,-0.005859,-0.312500,-0.182617,-0.397583,0.343750,...,0.131470,-0.093750,0.186523,0.722656,0.126953,0.287109,-0.093750,-0.271484,0.835938,-0.037354


In [40]:
##GENERATE WEIGHTS FOR PORTFOLIO TODAY
import pandas as pd
allocationsToStore = []
for portfolioKey in modelsInPortfolio:
    historicalWeights = None
    if portfolioTypes[portfolioKey] == "HRP FULL":
        print("HRP FULL")
        hrpReturns, historicalWeights = portfolioGeneration.produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 126, startIndex=3, maxWindowSize=False)
    elif portfolioTypes[portfolioKey] == "HRP WINDOW":
        print("HRP WINDOW")
        hrpReturns, historicalWeights = portfolioGeneration.produceHRPPredictions(aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]], 126, startIndex=3, maxWindowSize=True)
    elif portfolioTypes[portfolioKey] == "EW":
        print("EW")
        thisReturns = aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]]
        historicalWeights = pd.DataFrame(thisReturns.apply(lambda x: [1.0/len(x) for item in x], axis=1), columns=thisReturns.columns.values)
    elif portfolioTypes[portfolioKey] == "EW By Ticker":
        print("EW By Ticker")
        thisReturns = aggregateReturns[[portfolio.getModelHash(model) for model in modelsInPortfolio[portfolioKey]]]
        keptModels = []
        for mod in allModels:
            if portfolio.getModelHash(mod) in thisReturns.columns:
                keptModels.append(mod)
        weightArray = portfolioGeneration.getWeightingForAlgos(keptModels, thisReturns.columns)
        historicalWeights = pd.DataFrame(thisReturns.apply(lambda x: weightArray, axis=1), columns=thisReturns.columns.values)
    print(portfolioKey, historicalWeights.iloc[-1])
    todayWeight = historicalWeights.iloc[-1]
    netPosition = {}
    transformedAlgoPrediction = {}
    for model in modelsInPortfolio[portfolioKey]:
        if model.inputSeries.targetTicker not in netPosition:
            netPosition[model.inputSeries.targetTicker] = 0.0
        netPosition[model.inputSeries.targetTicker] += todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
        transformedAlgoPrediction[portfolio.getModelHash(model)] = todayWeight[portfolio.getModelHash(model)] * portfolio.getAggregatePredictionForModelDaily(model, joinedData)
    print(portfolioKey, netPosition)
    allocationsToStore.append(portfolioGeneration.storePortfolioAllocation(portfolioKey, portfolio.getToday(), todayWeight.to_dict(), netPosition, transformedAlgoPrediction, shouldReturn=True))
portfolio.storeManyItems(allocationsToStore)

EW By Ticker
443958fe526d5b10a120afb1acf292b2e21a9c112da6aa22bfd89d09 e5b77ac840e0694f124e4ceb80219238646ec98c98391057e8b5095e    0.018182
759581b497bcd8f07c66fede156ce5c6d1fe826b0192a98f030742a7    0.045455
7d0babac85bd5240804bca4fd99b925e30f55a7d299076ee21ce25ee    0.090909
b6bfc86cdf3ba24ff32db7f039190bace80978bb9c1b8fddae4175ad    0.090909
de51d3387dba4ff1c7191d771fff8acb8c714379633023cd249c2d0e    0.018182
3df5c4064d67b85ba4424dcc5a0a1f0e260b36cc5096855479beed22    0.045455
c155c65771efa17daaafbd3ee1c327d5b1843eade842874a01566d43    0.030303
fce63ca45cf23a15a747a745bb5abc7e4dd97ec9a9c7b2e203977006    0.030303
8f9d61400bc832f5b1af1cbac268de277f955a2aad6ae71c3e7fc4da    0.030303
43f88763edefd7af535eb11152055d8f83a70858286b6daa4319b127    0.090909
bc43e4c9a00c0e3749c0ba7906dc31a0bda35e01f302abf87aeb7d7a    0.018182
98f0c247c49553f0637560916983adb0645ada412ca8548f7ab70f89    0.090909
2083db1aa7ff59b8962603994801edc673c219bc1d1dfbd623522aaa    0.045455
bfb69268004322605e67a1d50f2565c84

In [41]:
##UPDATE CACHE
portfolioInfos = []
for portfolioHash in modelsInPortfolio:
    portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
    portfolioInfo = {
        "key":portfolioInfo.key.name,
        "description":portfolioInfo["description"],
        "benchmark":portfolioInfo["benchmark"],
        "portfolioType":portfolioInfo["portfolioType"],
        "startedTrading":portfolioInfo["startedTrading"]
    }
    print(portfolioInfo)
    portfolioInfos.append(portfolioInfo)

##GET ALL BENCHMARKS
benchmarksNeeded = []
print(joinedData.columns.values)
for info in portfolioInfos:
    print(info["benchmark"])
    if info["benchmark"] not in benchmarksNeeded and ("Adj_Close_" + info["benchmark"] not in joinedData.columns.values):
        benchmarksNeeded.append(info["benchmark"])
print("BENCHMARKS NEEDED", benchmarksNeeded)
pulledData, unused_ = dataAck.downloadTickerData(benchmarksNeeded)
thisJoinedData = joinedData
for ticker in pulledData:
    thisJoinedData = joinedData.join(pulledData[ticker], how='outer')

for info in portfolioInfos:
    portfolioHash = info["key"]
    print(portfolioHash)
    portfolioData = portfolioGeneration.getDataForPortfolio(portfolioHash, portfolioInfo["benchmark"], thisJoinedData, portfolioInfo["startedTrading"])
    portfolioGeneration.cachePortfolio(info, portfolioData, params.AVAILABLE_MODE)

{'key': '443958fe526d5b10a120afb1acf292b2e21a9c112da6aa22bfd89d09', 'description': 'Small Portfolio with Unique Models', 'benchmark': 'SPY', 'portfolioType': 'EW By Ticker', 'startedTrading': datetime.datetime(2017, 8, 18, 0, 0, tzinfo=<UTC>)}
{'key': '6ba6157d6c249c9e37d044751ffea3336d7523f1c18256af4ed45c2e', 'description': 'Small Portfolio with Unique Models', 'benchmark': 'SPY', 'portfolioType': 'HRP FULL', 'startedTrading': datetime.datetime(2017, 8, 18, 0, 0, tzinfo=<UTC>)}
['Adj_Close_XLV' 'Adj_Close_GLD' 'Adj_Close_LQD' 'Adj_Close_VBR'
 'Adj_Close_EEM' 'Adj_Close_KBE' 'Adj_Close_VNQ' 'Adj_Close_IWB'
 'Adj_Close_RSP' 'Adj_Close_FXE' 'Adj_Close_EWW' 'Adj_Close_SLV'
 'Adj_Close_EFA' 'Adj_Close_VGT' 'Adj_Close_IVE' 'Adj_Close_XBI'
 'Adj_Close_VTV' 'Adj_Close_IYR' 'Adj_Close_SDY' 'Adj_Close_IWR']
SPY
SPY
BENCHMARKS NEEDED ['SPY']
SPY
443958fe526d5b10a120afb1acf292b2e21a9c112da6aa22bfd89d09
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25)
((5, ('XBI', 12, None, 9, 2), 15, 'IVE'), 5,

In [42]:
## UPDATE CACHE FOR TRADING PORTFOLIOS
for mode in [params.PAPER_TRADING_MODE, params.TRADING_MODE]:
    print(mode)
    portfolioInfos = []
    downloadedPortfolioInfo = portfolioGeneration.getTradingPortfolioHashes(mode, includeDates = True)
    for tradingPortfolio in downloadedPortfolioInfo:
        portfolioHash = tradingPortfolio
        portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
        portfolioInfo = {
            "key":portfolioInfo.key.name,
            "description":portfolioInfo["description"],
            "benchmark":portfolioInfo["benchmark"],
            "portfolioType":portfolioInfo["portfolioType"],
            "startedTrading":downloadedPortfolioInfo[portfolioHash]
        }
        print(portfolioInfo)
        portfolioInfos.append(portfolioInfo)

    ##GET ALL BENCHMARKS
    benchmarksNeeded = []
    print(joinedData.columns.values)
    for info in portfolioInfos:
        print(info["benchmark"])
        if info["benchmark"] not in benchmarksNeeded and ("Adj_Close_" + info["benchmark"] not in joinedData.columns.values):
            benchmarksNeeded.append(info["benchmark"])
    print("BENCHMARKS NEEDED", benchmarksNeeded)
    pulledData, unused_ = dataAck.downloadTickerData(benchmarksNeeded)
    thisJoinedData = joinedData
    for ticker in pulledData:
        thisJoinedData = joinedData.join(pulledData[ticker], how='outer')
    for info in portfolioInfos:
        portfolioHash = info["key"]
        print(portfolioHash)
        portfolioData = portfolioGeneration.getDataForPortfolio(portfolioHash, portfolioInfo["benchmark"], thisJoinedData, portfolioInfo["startedTrading"])
        portfolioGeneration.cachePortfolio(info, portfolioData, mode)




PAPER_TRADING_MODE
{'key': '6ba6157d6c249c9e37d044751ffea3336d7523f1c18256af4ed45c2e', 'description': 'Small Portfolio with Unique Models', 'benchmark': 'SPY', 'portfolioType': 'HRP FULL', 'startedTrading': datetime.datetime(2017, 8, 18, 0, 0, tzinfo=<UTC>)}
['Adj_Close_XLV' 'Adj_Close_GLD' 'Adj_Close_LQD' 'Adj_Close_VBR'
 'Adj_Close_EEM' 'Adj_Close_KBE' 'Adj_Close_VNQ' 'Adj_Close_IWB'
 'Adj_Close_RSP' 'Adj_Close_FXE' 'Adj_Close_EWW' 'Adj_Close_SLV'
 'Adj_Close_EFA' 'Adj_Close_VGT' 'Adj_Close_IVE' 'Adj_Close_XBI'
 'Adj_Close_VTV' 'Adj_Close_IYR' 'Adj_Close_SDY' 'Adj_Close_IWR']
SPY
BENCHMARKS NEEDED ['SPY']
SPY
6ba6157d6c249c9e37d044751ffea3336d7523f1c18256af4ed45c2e
((22, ('GLD', 11, 1, 14, 1), 15, 'XLV'), 22, 25)
((5, ('SDY', 19, 18, None, 1), 7, 'IYR'), 5, 100)
((5, ('VBR', 2, None, None, 1), 7, 'LQD'), 5, 50)
((10, ('KBE', 29, 4, None, 3), 15, 'EEM'), 10, 25)
((5, ('XBI', 9, None, None, 1), 10, 'VGT'), 5, 25)
((22, ('FXE', 27, 8, None, 3), 7, 'RSP'), 22, 100)
((10, ('GLD', 11, 1, 1

# CALCULATE FUND PERFORMANCE METRICS

In [43]:
def cleanupAllocations(allocations):
    allocationsToReturn = {}
    for tradeDay in allocations:
        totalAllocation = sum([abs(allocations[tradeDay][ticker]) for ticker in allocations[tradeDay]])
        
        allocationsToReturn[tradeDay] = {}
        for ticker in allocations[tradeDay]:
            allocationsToReturn[tradeDay][ticker] = allocations[tradeDay][ticker]/totalAllocation
    
    return allocationsToReturn
        
    

In [44]:
import pandas as pd
def createAllocationsTable(scaledAllocations):
    allocationsTable = pd.DataFrame([])
    for allocationDay in scaledAllocations:
        keysInRow = list(scaledAllocations[allocationDay].keys())
        allocationsTable = pd.concat([allocationsTable, pd.DataFrame([[scaledAllocations[allocationDay][key] for key in keysInRow]], index = [allocationDay], columns=keysInRow).tz_localize(None)])
    return allocationsTable.sort_index().fillna(0)

In [45]:
from datetime import timedelta
def getNetAllocationAcrossPortfolios():
    portfolioInfos = []
    downloadedPortfolioInfo = portfolioGeneration.getTradingPortfolioHashes(params.TRADING_MODE, includeDates = True)
    for tradingPortfolio in downloadedPortfolioInfo:
        portfolioHash = tradingPortfolio
        portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
        portfolioInfo = {
            "key":portfolioInfo.key.name,
            "description":portfolioInfo["description"],
            "benchmark":portfolioInfo["benchmark"],
            "portfolioType":portfolioInfo["portfolioType"],
            "startedTrading":downloadedPortfolioInfo[portfolioHash]
        }
        print(portfolioInfo)
        portfolioInfos.append(portfolioInfo)
        
    if len(portfolioInfos) == 0:
        return None, None

    #DAILY ALLOCATIONS MUST ONLY GET ALLOCATIONS FOR PORTFOLIO AFTER START DATE FOR PORTFOLIO
    
    totalDesired = {}
    historicalAllocations = {}
    realizedAllocations = {}
    for tradingPortfolio in portfolioInfos:
        portfolioHash = tradingPortfolio["key"]
        allAllocations = portfolio.getPortfolioAllocations(portfolioHash)
        
        
        for item in allAllocations:
            netPosition = {}
            for key in item:
                if key.startswith("ticker_"):
                    netPosition[key[len("ticker_"):]] = item[key]
            
            ##REALIZED ALLOCATIONS
            if item["predictionDay"] >= tradingPortfolio["startedTrading"]:
                if item["predictionDay"] not in realizedAllocations:
                    realizedAllocations[item["predictionDay"]] = {}
                for ticker in netPosition:
                    if ticker not in realizedAllocations[item["predictionDay"]]:
                        realizedAllocations[item["predictionDay"]][ticker] = 0.0
                    realizedAllocations[item["predictionDay"]][ticker] += netPosition[ticker]
            
            if item["predictionDay"] not in historicalAllocations:
                historicalAllocations[item["predictionDay"]] = {}
            for ticker in netPosition:
                if ticker not in historicalAllocations[item["predictionDay"]]:
                    historicalAllocations[item["predictionDay"]][ticker] = 0.0
                historicalAllocations[item["predictionDay"]][ticker] += netPosition[ticker]
    
    
    ##CLEANUP ALLOCATIONS TO SCALE TO CAPITAL 1
    realizedAllocations = createAllocationsTable(cleanupAllocations(realizedAllocations))
    historicalAllocations = createAllocationsTable(cleanupAllocations(historicalAllocations))       
    return historicalAllocations, realizedAllocations
    

In [46]:
import empyrical
import json
import portfolioGeneration
def getFundData():
    historicalAllocations, realizedAllocations =  getNetAllocationAcrossPortfolios()
    if historicalAllocations is None:
        return None, None
    pulledData, unused_ = dataAck.downloadTickerData(historicalAllocations.columns.values)
    allocationJoinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
    dataToCache = []
    for allocationForm in [historicalAllocations, realizedAllocations]:
        performanceByTicker, fundPerformance, fundTransactionCost = portfolioGeneration.calculatePerformanceForAllocations(allocationForm, allocationJoinedData)
        if len(fundPerformance) == 0:
            dataToCache.append({})
            continue
        
        ##CALCULATE BETAS FOR ALL TICKERS TO FUND PERFORMANCE
        tickerAlphaBetas = []
        for ticker in allocationForm.columns.values:
            factorReturn = dataAck.getDailyFactorReturn(ticker, allocationJoinedData)
            alpha, beta = empyrical.alpha_beta(fundPerformance, factorReturn)
            tickerAlphaBetas.append({"ticker":ticker, "alpha":alpha * 100, "beta":beta})
        
        tickerCols, tickerRows = portfolioGeneration.convertTableToJSON(empyrical.cum_returns(performanceByTicker))
        tickerAllocationsCols, tickerAllocationsRows = portfolioGeneration.convertTableToJSON(allocationForm)
        fundCols, fundRows = portfolioGeneration.convertTableToJSON(empyrical.cum_returns(fundPerformance))
        
        sharpe = empyrical.sharpe_ratio(fundPerformance)
        annualReturn = empyrical.annual_return(fundPerformance)[0]
        annualVol = empyrical.annual_volatility(fundPerformance)
        
        commissionCols, commissionRows = portfolioGeneration.convertTableToJSON(fundTransactionCost)
        
        
        dataToCache.append({
            "tickerAlphaBetas":tickerAlphaBetas,
            "tickerCols":json.dumps(tickerCols),
            "tickerRows":json.dumps(tickerRows),
            "tickerAllocationsCols":json.dumps(tickerAllocationsCols),
            "tickerAllocationsRows":json.dumps(tickerAllocationsRows),
            "fundCols":json.dumps(fundCols),
            "fundRows":json.dumps(fundRows),
            "sharpe":sharpe,
            "annualReturn":annualReturn * 100,
            "annualVol":annualVol * 100,
            "commissionCols":json.dumps(commissionCols),
            "commissionRows":json.dumps(commissionRows)  
            
        })
    
    historicalData = dataToCache[0]
    realizedData = dataToCache[1]
    ##GET TODAY ALLOCATION
    if realizedData != {}:
        newRows = []
        tARows = json.loads(realizedData["tickerAllocationsRows"])
        tACols = json.loads(realizedData["tickerAllocationsCols"])
        print(tARows[-1])
        for i in range(len(tACols)):
            
            newRows.append([tACols[i], abs(tARows[-1][i + 1])]) ##i+1 because date
        realizedData["todayAllocation"] = json.dumps(newRows)
        print(realizedData["todayAllocation"])
    
    return historicalData, realizedData

In [47]:
##UPLOAD SO EASY TO READ
import pickle
from google.cloud import datastore, storage, logging
def cacheFundData(historicalData, realizedData):
    storageClient = storage.Client('money-maker-1236')
    while True:
        try:
            bucket = storageClient.get_bucket(params.portfolioDataTradingCache) ##JUST SAVE IN TRADING BUCKET
            blob = storage.Blob("HISTORICALFUND", bucket)
            blob.upload_from_string(pickle.dumps(historicalData))
            break
        except:
            print("UPLOAD BLOB ERROR:", str(sys.exc_info()))
            time.sleep(10)
    while True:
        try:
            bucket = storageClient.get_bucket(params.portfolioDataTradingCache) ##JUST SAVE IN TRADING BUCKET
            blob = storage.Blob("REALIZEDFUND", bucket)
            blob.upload_from_string(pickle.dumps(realizedData))
            break
        except:
            print("UPLOAD BLOB ERROR:", str(sys.exc_info()))
            time.sleep(10)
    
    

In [48]:
historicalData, realizedData = getFundData()
if historicalData is not None:
    cacheFundData(historicalData, realizedData)

# LOG COMPLETED TIME

In [49]:
portfolioGeneration.storeSystemStatus(portfolio.getToday(), startTime, datetime.datetime.now(),\
                str(datetime.datetime.now() - startTime), modelsSeen=modelsSeen, portfoliosSeen=portfoliosSeen)